In [27]:
import numpy as np
import cvxpy as cp

# Dimension of vectors xi, v, and x
C = 5
                
np.random.seed(41)  
for i in range(20):
    xi = np.sort(np.random.random(C))
    v = np.linspace(0, 1, C, endpoint=False)
    w = np.random.choice(v)
    
    # cvxpy solution with equality constraint (no pruning)
    x_no_pruning = cp.Variable(C)
    equality_constraint_no_pruning = v @ x_no_pruning == w
    constraints_no_pruning = [
        equality_constraint_no_pruning,
        cp.sum(x_no_pruning) == 1,
        x_no_pruning >= 0,
        x_no_pruning <= 1
    ]
    objective_no_pruning = cp.Minimize(xi @ x_no_pruning)
    prob_no_pruning = cp.Problem(objective_no_pruning, constraints_no_pruning)
    prob_no_pruning.solve()
    
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve()
    
    print(f"\nIteration: {i}")
    print("DATI:")
    print("xi:", np.array2string(xi, precision=2, separator=', '))
    print("v:", np.array2string(v, precision=2, separator=', '))
    print("w:", w, "\n")

    # Handling cases where the problem has no solution
    if prob_no_pruning.value is not None and x_no_pruning.value is not None:
        print(f"Objective value (no pruning): {prob_no_pruning.value:.2f}")
        print("Optimal x (no pruning):", 
              np.array2string(x_no_pruning.value, precision=2, separator=', ', suppress_small=True))
        if equality_constraint_no_pruning.dual_value is not None:
            print(f"Lagrange multiplier (no pruning): {equality_constraint_no_pruning.dual_value:.2f}")
    else:
        print("No solution found for the problem without pruning.")
    print()
    if prob_pruning.value is not None and x_pruning.value is not None:
        print(f"Objective value (pruning): {prob_pruning.value:.2f}")
        print("Optimal x (pruning):", 
              np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))
        if equality_constraint_pruning.dual_value is not None:
            print(f"Lagrange multiplier (pruning): {equality_constraint_pruning.dual_value:.2f}")
    else:
        print("No solution found for the problem with pruning.")
    
    print("-"*40)



Iteration: 0
DATI:
xi: [0.04, 0.05, 0.12, 0.25, 0.68]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: 0.4 

Objective value (no pruning): 0.12
Optimal x (no pruning): [0., 0., 1., 0., 0.]
Lagrange multiplier (no pruning): -0.56

Objective value (pruning): 0.12
Optimal x (pruning): [0., 0., 1., 0., 0.]
Lagrange multiplier (pruning): -0.60
----------------------------------------

Iteration: 1
DATI:
xi: [0.7 , 0.76, 0.79, 0.8 , 0.83]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: 0.2 

Objective value (no pruning): 0.74
Optimal x (no pruning): [0.75, 0.  , 0.  , 0.  , 0.25]
Lagrange multiplier (no pruning): -0.16

Objective value (pruning): 0.21
Optimal x (pruning): [0.  , 0.  , 0.  , 0.  , 0.25]
Lagrange multiplier (pruning): -1.04
----------------------------------------

Iteration: 2
DATI:
xi: [0.24, 0.46, 0.51, 0.7 , 0.87]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: 0.4 

Objective value (no pruning): 0.51
Optimal x (no pruning): [0., 0., 1., 0., 0.]
Lagrange multiplier (no pruning): -0.76

Objective value (pruning): 0.44
Op